In [1]:
#imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, load_img, img_to_array
import numpy as np

from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
class Metrics(keras.callbacks.Callback):
    """
    Implementation of custom metrics: Precision, Recall, F-Measure and Confusion Matrix
    """
    
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        
        for i in range(len(self.validation_data)):
            x_val, y_val = self.validation_data.__getitem__(i)

        print(y_val.shape)
        print(y_val[1])
        print(y_val[2])

        y_predict = np.asarray(model.predict(self.validation_data, steps = 1))
        print(y_predict.shape)

        with tf.Session() as sess:
            lab = [i for i in range(0,75)]
            print(y_val.shape)

            y_val = np.argmax(y_val, axis=1)
            y_predict = np.argmax(y_predict, axis=1)
            print("y_val: ", y_val)
            print("y_predict:", y_predict)
            print("\nMetrics for Epoch")
            print("Confusion Matrix:\n",confusion_matrix(y_val,y_predict, labels=lab))
            print("Recall: ", recall_score(y_val,y_predict, average=None, labels=lab))
            print("Precision: ", precision_score(y_val,y_predict, average = None, labels=lab))
            print("F1_score: ", f1_score(y_val,y_predict, average =None, labels=lab))
            print("\n") 
            self._data.append({
                'val_recall': recall_score(y_val, y_predict, average = None, labels=lab),
                'val_precision': precision_score(y_val, y_predict, average = None, labels=lab),
                'val_f1_score': f1_score(y_val,y_predict, average = None, labels=lab),
            })
            return

    def get_data(self):
        return self._data
    
metrics = Metrics()

In [3]:
#Data Generator for Data augmentation. Edit possible
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.0017)

In [4]:
#Sanity check for augmentation

img = load_img('../datasets/large_sample/almond_desert/6350.jpg')  # this is a PIL image
img=img.resize((28,28))
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
print(x.shape)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='almond_desert', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

(28, 28, 3)


In [5]:
train_generator = train_datagen.flow_from_directory(
        '../datasets/large_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'training')  # since we use binary_crossentropy loss, we need binary labels



Found 45815 images belonging to 75 classes.


In [6]:
val_generator = train_datagen.flow_from_directory(
        '../datasets/large_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=27,
        class_mode='categorical',
        subset = 'validation')  # since we use binary_crossentropy loss, we need binary labels

Found 27 images belonging to 75 classes.


In [7]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

{0: 'almond_desert', 1: 'apple_pine', 2: 'baby_cabbage', 3: 'breakfast_burritos', 4: 'cake', 5: 'candied_haw', 6: 'chinese_mitten_crab', 7: 'clam_juice_chicken_feet', 8: 'coconut_cake', 9: 'coffee', 10: 'cool_noodles', 11: 'corn_cake', 12: 'donuts', 13: 'double_cooked_pork_slices', 14: 'dry_fried_noodle', 15: 'duck_neck', 16: 'dumpling_ham', 17: 'dumplings', 18: 'egg_pudding', 19: 'egg_tart', 20: 'egg_waffle', 21: 'egg_yolk_puff', 22: 'ferment_cake', 23: 'french_toast', 24: 'fried_dough_twists', 25: 'fried_rice', 26: 'fruit_salad', 27: 'grilled_cold_noodles', 28: 'grilled_fish', 29: 'gui_ling_jelly', 30: 'hong_kong_style_french_toast', 31: 'hot_and_sour_soup', 32: 'hot_pot', 33: 'huntun', 34: 'ice_cream', 35: 'mango_crepe', 36: 'mashed_potato', 37: 'milk_pudding', 38: 'millet_porridge', 39: 'mixed_porridge', 40: 'moon_cake', 41: 'pancake', 42: 'pickle', 43: 'pineapple_buns', 44: 'pizza', 45: 'pork_knuckle', 46: 'potato_shred', 47: 'puffs', 48: 'pumpkin_porridge', 49: 'red_bean_pudding'

In [8]:
#Naive Model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(75))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [9]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
__________

In [10]:
model.fit_generator(train_generator,
                   steps_per_epoch = 1,
                   epochs = 1, validation_data = val_generator, callbacks=[metrics])

Epoch 1/1
(27, 75)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
(27, 75)
(27, 75)
y_val:  [66 43  5 51 40 29 63 24 69  6 39 25 37  2 71 14 16 58 59 53 67 48  1 50
 26 65 44]
y_predict: [60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 66 60 60 60
 60 60 60]

Metrics for Epoch
Confusion Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Recall:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.

/home/mudit/anaconda3/envs/TensorflowGPU/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/mudit/anaconda3/envs/TensorflowGPU/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mudit/anaconda3/envs/TensorflowGPU/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mudit/anaconda3/envs/TensorflowGPU/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', 

In [11]:
img = load_img('../datasets/large_sample/almond_desert/6350.jpg')  # this is a PIL image
img=img.resize((28,28))
display(img)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

(1, 28, 28, 3)


In [12]:

y = model.predict(val_generator, steps = 1)

In [13]:
labels[np.argmax(y[1])]

'sliced_cucumber'

In [14]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 conv2d
1 activation
2 max_pooling2d
3 conv2d_1
4 activation_1
5 max_pooling2d_1
6 conv2d_2
7 activation_2
8 max_pooling2d_2
9 flatten
10 dense
11 activation_3
12 dropout
13 dense_1
14 activation_4


In [24]:
#Data Generator for Data augmentation. Edit possible
val_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.3)

In [25]:
small_train_generator = val_datagen.flow_from_directory(
        '../datasets/small_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'training')

Found 100 images belonging to 25 classes.


In [26]:
small_val_generator = val_datagen.flow_from_directory(
        '../datasets/small_sample',  # this is the target directory
        target_size=(28, 28),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical',
        subset = 'validation')

Found 25 images belonging to 25 classes.


In [27]:
for layer in model.layers[:13]:
    layer.trainable=False


In [28]:
for i,layer in enumerate(model.layers):
  print(i,layer.trainable)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 True
14 True


In [29]:
model.pop()
model.add(Dense(25))
model.add(Activation('sigmoid'))

In [30]:

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
__________

In [32]:
model.fit_generator(small_train_generator,
                   steps_per_epoch = 100,
                   epochs = 1, validation_data = small_val_generator)

Epoch 1/1
100/100 [==============================] - 13s 125ms/step - loss: 3.2225 - acc: 0.0347 - val_loss: 3.2213 - val_acc: 0.0400


In [33]:
img = load_img('../datasets/small_sample/black_rice_cake/10585.jpg')  # this is a PIL image
img=img.resize((28,28))
display(img)
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

(1, 28, 28, 3)


In [34]:
labelsSmall = (small_train_generator.class_indices)
labelsSmall = dict((v,k) for k,v in labelsSmall.items())
print(labelsSmall)

{0: 'Omelette', 1: 'Snowflake_Flaky_Pastry', 2: 'black_rice_cake', 3: 'black_sesame_paste', 4: 'boiled_corn', 5: 'braised_pork', 6: 'cheese', 7: 'claypot', 8: 'french_fries', 9: 'fried_chestnuts', 10: 'frozen_lemon_tea', 11: 'glutinous_rice_ball_with_peanut_and_sesame_sweets', 12: 'glutinous_rice_dumpling', 13: 'green_bean_cake', 14: 'hot_dog', 15: 'layer_cake', 16: 'mango_pomelo_sago', 17: 'onion_rings', 18: 'persimmon_pancake', 19: 'poon_choi', 20: 'pop_corn', 21: 'sampan_porridge', 22: 'scrambled_eggs_with_tomatoes', 23: 'steak', 24: 'stir_fried_lettuce'}


In [35]:
y = model.predict(x)

In [36]:
labels[np.argmax(y)]

'baby_cabbage'